In [ ]:
import pandas as pd

ohlcv_data = pd.read_csv("/Users/newuser/Projects/robust_algo_trader/data/gen_oanda_data/GBP_USD_M15_raw_data.csv", parse_dates=['time'])
ohlcv_data = ohlcv_data.set_index('time')
ohlcv_data['year'] = ohlcv_data.index.year
ohlcv_data['month'] = ohlcv_data.index.month
ohlcv_data['day_of_week'] = ohlcv_data.index.dayofweek
ohlcv_data['hour'] = ohlcv_data.index.hour
ohlcv_data['minute'] = ohlcv_data.index.minute
start_date = '2007-01-01'
end_date = '2015-01-01'
ohlcv_data = ohlcv_data[start_date:end_date]
df = ohlcv_data.copy()
df

In [ ]:
# Convert the 'Date' column to datetime
ohlcv_data['Date'] = pd.to_datetime(ohlcv_data['Date'])

# Set the 'Date' column as the index
ohlcv_data.set_index('Date', inplace=True)

# Define the start date and the duration for the data extraction
start_date = '2007-01-01'
duration_months = 6

# Calculate the end date by adding the duration to the start date
end_date = pd.to_datetime(start_date) + pd.DateOffset(months=duration_months)

# Extract the data between the start date and the end date
six_months_data = ohlcv_data.loc[start_date:end_date]

# Display the extracted data
print(six_months_data)

In [ ]:
df

In [ ]:
from sktime.forecasting.model_selection import SlidingWindowSplitter

ONE_DAY = 4*24   
train_size = 80 * ONE_DAY # 80 days 
test_size = 10 * ONE_DAY # 10 days
val_test_horizon = test_size * 2
splitter = SlidingWindowSplitter(window_length=train_size, 
                                 fh=np.arange(1, val_test_horizon + 1),
                                 step_length=test_size)

for i, (train_idx, val_test_idx) in enumerate(splitter.split(df)):
    val_idx = val_test_idx[:test_size]
    test_idx = val_test_idx[test_size:]
    
    df_train = df.iloc[train_idx, :].to_numpy()
    df_val = df.iloc[val_idx, :].to_numpy()
    df_test = df.iloc[test_idx, :].to_numpy()
    
    print(f"Window: {i}")
    print(f"Train: {df_train.shape[0]}")
    print(f"Val: {df_val.shape[0]}")
    print(f"Test: {df_test.shape[0]}")
    print()